## Importing Libraries

Importing the necessary libraries for the project

And initializing constants

```python

```


In [ ]:
!python -m venv venv

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
%pip install opencv-python-headless numpy ultralytics Pillow ipywidgets wandb

In [ ]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps
import wandb

In [ ]:
### CONSTANTS
# Image properties
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
IMAGE_CHANNELS = 3

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.gif')


# Director paths
PROCESSED_DS_PATH_NM = './processed_dataset/NoMask'
PROCESSED_DS_PATH_M = './processed_dataset/Mask'
RAW_DS_PATH = './Data'

## MODELS
# Segmentation
SEG_KWARGS = {
    'classes':  0,
    'imgsz': IMAGE_WIDTH,
    'boxes':True,
    'save':True,
    'show_labels':False,
    'show_conf':False,
    'max_det':1,
}
# Classification
CLASS_KWARGS = {
    'epochs': 100,
    'project': 'models',
}
MODEL_KWARGS = {
    'save': True,
    'project': './',
    'name': 'tests',
}

## Preparig data for training


### Preparing Models

```python

```


In [ ]:
img_seg_model = YOLO('yolov8n-seg.pt')
img_class_model = YOLO("yolov8s-cls.pt")

### Preparing dataset

```python

```


In [ ]:
def process_images(
        source_folder=RAW_DS_PATH, 
        destination_folder=PROCESSED_DS_PATH_NM, 
        grayscale=True, 
        resize=True,
        resize_dimensions=IMAGE_SIZE):
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    os.makedirs(destination_folder, exist_ok=True)

    for subdir, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(IMAGE_EXT):
                source_path = os.path.join(subdir, file)
                relative_path = os.path.relpath(subdir, source_folder)
                destination_subdir = os.path.join(destination_folder, relative_path)
                os.makedirs(destination_subdir, exist_ok=True)

                destination_path = os.path.join(destination_subdir, file)

                image = cv2.imread(source_path)
                if image is None:
                    print(f"Unable to read image: {source_path}")
                    continue

                if grayscale:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                if resize:
                    image = cv2.resize(image, resize_dimensions)

                cv2.imwrite(destination_path, image)

In [ ]:
def mask_images(
        source_folder = PROCESSED_DS_PATH_NM,
        destination_folder=PROCESSED_DS_PATH_M):
    
    if not os.path.exists(source_folder):
        print(f"Source folder does not exist: {source_folder}")
        return
    if os.path.exists(destination_folder):
        print(f"Destination folder already exists: {destination_folder}")
        return

    root_path_length = len(source_folder) + len(os.path.sep)

    # Set comprehension to get only directories at depth 2
    subdirs = {subdir[root_path_length:] for subdir, dirs, files in os.walk(source_folder)
               if subdir.count(os.path.sep) == 2}
    
    for subdir in subdirs:
        img_folder = os.path.join(source_folder, subdir)
        print(img_folder)
        img_seg_model(img_folder, project=destination_folder, name=subdir, **SEG_KWARGS)

In [ ]:
process_images()

mask_images()

## Training

```python

```


In [ ]:
wandb.init(mode='disabled');

try:

    run = max(os.listdir(CLASS_KWARGS['project']))

    run = 1 + int(run)

except:

    run = 1


img_class_model.train(data=PROCESSED_DS_PATH_M, name=run, **CLASS_KWARGS)

## Prediction


In [ ]:
def get_model(run, best=False):
    weight = 'weights/best.pt' if best else 'weights/last.pt'
    path = os.path.join(CLASS_KWARGS['project'], str(run), weight)
    return YOLO(path)

In [ ]:
def predict(image_path):
    try:
        runs = os.listdir(CLASS_KWARGS['project'])
    except:
        print("No runs found")
        return
    i = input(f"Choose a run from {runs}: ")

    while i not in runs:
        i = input(f"Choose a run from {runs}: ")
    
    model = get_model(i)

    model(image_path, **MODEL_KWARGS)

In [ ]:
predict('./processed_dataset/Mask/val/6/')